# ch06 커맨드 패턴으로 호출 캡슐화하기

- 커맨드 디자인 패턴으로 어떻게 실행 취소와 매크로 동작을 구현하는지 배우고, 실행 후에 취소할 수 있는 간단한 유닉스 명령어 변형을 작성해봄
- 프린터 프로그램을 만들고 있고 프린터 스풀러(spooler)를 구현하고 싶다고 가정

### Spooler Logic

1. Spooler 클래스를 만들고 
2. 프린터 작업을 추가하거나 제거하는 메소드를 추가


### 프린터 작업을 실행하는 가장 쉬운 방법

1. 우선 프린트할 텍스트, 부수, 색상, 질 등 모든 정보를 포함한 객체를 만드는 것
2. 스풀러는 프린트 작업의 execute 메소드를 호출해야 하고, 프린트 작업은 스스로 이를 처리함

- 커맨드 패턴이 동작하는 원리: 메소드를 호출할 모든 정보를 캡슐화하고 있는 객체를 하나 만듦
- 메소드 이름, 메소드를 소유하는 객체, 메소드 파라미터 값 등이 포함됨
- 커맨드 객체는 execute 메소드를 호출하고, 객체를 저장하고, 다른 객체 형태로 반환할 줄 아는 모든 코드에 전달할 수 있음
- '프록시와 옵저버 패턴으로 객체 간 통신하기' 에서 이벤트 알림을 받는 것이 주업무인 옵저버 패턴에 대해서 배움
- 커맨드 패턴을 쓰면, 커맨드 인스턴스를 옵저버로서 전달할 수 있고, 서브젝트가 옵저버에 이벤트 알림을 보낼 때 커맨드 인스턴스를 호출하고 캡슐화 해 둔 작업을 마칠 수 있음

## 커맨드 패턴 용어

- Command: 작업을 실행하기 위한 인터페이스
- ConcreteCommand: 이 클래스는 Command 인터페이스를 확장하고 execute 메소드를 구현함. 이 클래스는 액션과 리시버 사이의 바인딩을 생성함
- Client: 이 클래스는 ConcreteCommand 클래스를 생성하고 리시버와 연동함
- Invoker: 이 클래스는 커맨드가 요청을 수행하도록 함
- Receiver: 이 클래스는 동작을 어떻게 수행해야 할지 알고 있음


- 커맨드 디자인 패턴은 어떤 작업을 수행하기 위한 메소드를 호출하는 인터페이스를 제공하고, 한 번의 객체 호출로 작업을 마치기 위해 필요한 정보를 캡슐화해서 인스턴스화 한 후에 별도로 실행할 수 있게 함

## 커맨드 디자인 패턴의 사용 예

- 요청의 히스토리를 기억해야 할 때: invoker는 execute 메소드를 호출한 후에 커맨드 인스턴스를  저장해서 히스토리 기능을 구현할 수 있음
- 콜백 기능을 구현해야 할 때: 인보커에 두 객체를 차례로 전달하면, 두번째 객체는 첫번째 객체의 콜백이 됨
- 여러 시간이나 순서로 요청을 처리해야 할 때: 이렇게 하려면 커맨드 객체를 서로 다른 인보커에 전달해서 다른 조건으로 실행되게 해야함
- 실행을 처리하는 객체로부터 인보커를 분리해야 할 때
- 실행 취소 기능을 구현할 때: 이를 구현하려면 execute 메소드에서 수행한 작업을 취소하는 메소드를 정의해야 함. 예를 들어 파일을 생성했다면 이 파일을 삭제해야 함

## 커맨드 디자인 패턴의 장단점

- 구조체를 만들 때, 특히 요청과 실행이 서로에 의존하지 않을 때 유용. Command 인스턴스는 Client가 인스턴스화할 수 있지만 실행은 추후에 Invoker가 하고, Client와 Invoker는 서로를 모를 수 있음
- 기존 코드를 수정하지 않고 새로운 커맨드를 추가할 수 있어 확장성이 좋음
- 매크로라 불리는 연속된 커맨드를 만들 수 있음. 매크로를 실행하려면 Command 리스트를 만들고 모든 커맨드에 대해 execute 메소드를 호출함
- 중요한 단점: 단 하나의 커맨드에 대해 클래스가 많아진다는 점

## 파이썬 구현

- ls, touch, rm과 같이 아주 간단한 유닉스 명령어를 만들어 봄
- 쉘의 중요한 기능: 쉘이 시작한 이후 실행한 모든 동작을 취소할 수 있다는 것
- 메소드를 추상으로 만들기 위해서 abstractmethod 데코레이터를 제공하는 abc 모듈을 사용함
- 따라서 이 메소드는 하위 클래스를 인스턴스화하기 위해 반드시 구현해야 함
- 또한 \_\_metaclass\_\_ 클래스 속성에 abc.ABCMeta를 할당해서 abstractmethod 데코레이터가 동작하게 해야 함

In [ ]:
import abc
import os

history = []


class Command(object):
    """커맨드 인터페이스"""
    
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def execute(self):
        """커맨드를 실행하기 위한 메소드"""
        pass

In [12]:
class LsCommand(Command):
    """유닉스 명령어 ls를 흉내내는 실제 커맨드"""
    def __init__(self, receiver):
        self.receiver = receiver
        
    def execute(self):
        """리시버로의 호출을 델리게이트하는 커맨드"""
        self.receiver.show_current_dir()
        
    def undo(self):
        """ls 커맨드는 취소할 수 없다"""
        pass
    
    
class LsReceiver(object):
    def show_current_dir(self):
        """리시버는 어떻게 커맨드를 실행해야 하는지 알고 있다"""
        cur_dir = './'
        
        filenames = []
        for filename in os.listdir(cur_dir):
            if os.path.isfile(os.path.join(cur_dir, filename)):
                filenames.append(filename)
                
        print('Content of dir: ', ' '.join(filenames))

In [13]:
os.utime?

In [14]:
class TouchCommand(Command):
    """유닉스 명령어 touch를 흉내내는 실제 명령어 동작"""
    def __init__(self, receiver):
        self.receiver = receiver
        
    def execute(self):
        self.receiver.create_file()
        
    def undo(self):
        self.receiver.delete_file()
        

class TouchReceiver(object):
    def __init__(self, filename):
        self.filename = filename
        
    def create_file(self):
        """유닉스 touch 명령어의 실제 구현"""
        with open(self.filename, 'a'):
            os.utime(self.filename, None)
        
    def delete_file(self):
        """유닉스 명령어 touch 실행 취소. 여기선 간단히 파일을 삭제한다."""
        os.remove(self.filename)

In [16]:
class RmCommand(Command):
    """유닉스 명령어 rm를 흉내내는 실제 명령어 동작"""
    def __init__(self, receiver):
        self.receiver = receiver
        
    def execute(self):
        self.receiver.delete_file()
        
    def undo(self):
        self.receiver.undo()
        
        
class RmReceiver(object):
    def __init__(self, filename):
        self.filename = filename
        self.backup_name = None
        
    def delete_file(self):
        """백업을 만드는 식으로 파일을 삭제하고 실행 취소 메소드에 저장한다."""
        self.backup_name = '.' + self.filename
        os.rename(self.filename, self.backup_name)
        
    def undo(self):
        """삭제한 파일을 되살린다."""
        original_name = self.backup_name[1:]
        os.rename(self.backup_name, original_name)
        self.backup_name = None

- 인보커를 보자
- 명령어의 리스트를 받아서 파일을 생성하고 삭제한다.
- 파일을 생성하거나 삭제할 때, 모든 명령어는 차례대로 실행된다.
- 명령어를 실행할 때 인보커에서 히스토리를 가지고 있다는 점을 기억해야 함
- 이 히스토리는 추후 실행 취소를 구현할 때 필요함
- 명령을 실행 취소하려고 할 때 실행했던 명령어 리스트를 가져옴
- 마지막에, 인보커가 이미 실행했던 명령어의 undo 메소드를 호출함

In [22]:
class Invoekr(object):
    def __init__(self, create_file_commands, delete_file_commands):
        self.create_file_commands = create_file_commands
        self.delete_file_commands = delete_file_commands
        self.history = []
        
    def create_file(self):
        print('Creating file...')
        for command in self.create_file_commands:
            command.execute()
            self.history.append(command)
            
        print('File created.\n')
        
    def delete_file(self):
        print('Deleting file...')
        for command in self.delete_file_commands:
            command.execute()
            self.history.append(command)
        print('File deleted.\n')
        
    def undo_all(self):
        print('Undo all...')
        for command in reversed(self.history):
            command.undo()
        print('Undo all finished.')

In [23]:
if __name__ == '__main__':
    # 클라이언트
    # 현재 디렉토리 파일을 표시한다.
    ls_receiver = LsReceiver()
    ls_command = LsCommand(ls_receiver)
    
    # 파일 생성
    touch_receiver = TouchReceiver('test_file')
    touch_command = TouchCommand(touch_receiver)
    
    # 생성한 파일 삭제
    rm_receiver = RmReceiver('test_file')
    rm_command = RmCommand(rm_receiver)
    
    create_file_commands = [ls_command, touch_command, ls_command]
    delete_file_commands = [ls_command, rm_command, ls_command]
    
    invoker = Invoekr(create_file_commands, delete_file_commands)
    
    invoker.create_file()
    invoker.delete_file()
    invoker.undo_all()

Creating file...
('Content of dir: ', '__init__.py ch01.ipynb ch02.ipynb ch03.ipynb ch04.ipynb ch05.ipynb ch06.ipynb crawler.py last_short.p myfile short_to_url.p')
('Content of dir: ', '__init__.py ch01.ipynb ch02.ipynb ch03.ipynb ch04.ipynb ch05.ipynb ch06.ipynb crawler.py last_short.p myfile short_to_url.p test_file')
File created.

Deleting file...
('Content of dir: ', '__init__.py ch01.ipynb ch02.ipynb ch03.ipynb ch04.ipynb ch05.ipynb ch06.ipynb crawler.py last_short.p myfile short_to_url.p test_file')
('Content of dir: ', '.test_file __init__.py ch01.ipynb ch02.ipynb ch03.ipynb ch04.ipynb ch05.ipynb ch06.ipynb crawler.py last_short.p myfile short_to_url.p')
File deleted.

Undo all...
Undo all finished.


## 요약

- 커맨드 디자인 패턴은 어떤 작업을 수행하기 위한 메소드를 호출하는 인터페이스를 제공하고, 한 번의 객체 호출로 작업을 마치는 데 필요한 정보를 캡슐화해서 인스턴스화 한 후에 별도로 실행할 수 있게 함
- 기존에 실행한 함수의 결과를 취소하는 메소드를 구현해 놓았다면, 실행 취소 기능을 만들 때 커맨드 디자인 패턴을 사용할 수 있음
- 커맨드 디자인 패턴은 실행했던 동작의 히스토리를 구현할 때나, Command 인스턴스 집합으로써 매크로를 만들어 차례대로 실행할 때도 사용할 수 있음
- 최소한의 수정으로 알고리즘을 수정하게 해주는 Template Method 패턴에 대해 배움